In [405]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

In [406]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(700, 700, "Cilindro", None, None)
glfw.make_context_current(window)

In [407]:
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

In [408]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

In [409]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


In [410]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

In [411]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


In [412]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

In [413]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


In [414]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

In [415]:
import math


PI = 3.141592
num_sectors = 30 # qtd de sectors (longitude)
num_stacks = 30 # qtd de stacks (latitude)

num_vertices = 64 # define a "qualidade" do circulo
counter = 0
r = 0.25

vertices_list = []

angle = 0.0
for counter in range(num_vertices):
    angle += 2*PI/num_vertices 
    x = math.cos(angle)*r
    y = math.sin(angle)*r
    vertices_list.append([x,y,0.5])

for counter in range(num_vertices*2):
    angle += 2*PI/num_vertices 
    x = math.cos(angle)*r
    y = math.sin(angle)*r
    vertices_list.append([x,y,0.0])


# vertices_list = []
# grid sectos vs stacks (longitude vs latitude)
sector_step=(PI*2)/num_sectors # variar de 0 até 2π
stack_step=(0.5)/num_stacks # variar de 0 até π


# Entrada: angulo de longitude, latitude, raio
# Saida: coordenadas no cilindro
def F(u,v,r):
    x = r*math.cos(u)
    y = r*math.sin(u)
    z = v
    return (x,y,z)

# vamos gerar um conjunto de vertices representantes poligonos
# para a superficie da esfera.
# cada poligono eh representado por dois triangulos

for i in range(0,num_sectors): # para cada sector (longitude)
    for j in range(0,num_stacks): # para cada stack (latitude)
        
        u = i * sector_step # angulo setor
        v = j * stack_step # angulo stack
        
        un = 0 # angulo do proximo sector
        if i+1==num_sectors:
            un = PI*2
        else: un = (i+1)*sector_step
            
        vn = 0 # angulo do proximo stack
        if j+1==num_stacks:
            vn = 0.5
        else: vn = (j+1)*stack_step
        
        # verticies do poligono
        p0=F(u, v, r)
        p1=F(u, vn, r)
        p2=F(un, v, r)
        p3=F(un, vn, r)
        
        # triangulo 1 (primeira parte do poligono)
        vertices_list.append(p0)
        vertices_list.append(p2)
        vertices_list.append(p1)
        
        # triangulo 2 (segunda e ultima parte do poligono)
        vertices_list.append(p3)
        vertices_list.append(p1)
        vertices_list.append(p2)


total_vertices = len(vertices_list)
vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
vertices['position'] = np.array(vertices_list)

In [416]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


In [417]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

In [418]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


In [419]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

In [420]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

In [421]:
loc_color = glGetUniformLocation(program, "color")

In [422]:

s_x = 1.0
s_y = 1.0
s_z = 1.0

t_x = 0.0
t_y = 0.0
t_z = 0.0

stopRotation = 0

def key_event(window,key,scancode,action,mods):
    global t_x, t_y, t_z, s_x, s_y, s_z, stopRotation
    if key == 87: t_y += 0.01 #cima W
    if key == 83: t_y -= 0.01 #baixo S
    if key == 65: t_x -= 0.01 #esquerda A
    if key == 68: t_x += 0.01 #direita D
    if key == 69: t_z += 0.01 #eixo z (tecla E)
    if key == 81: t_z -= 0.01 #eixo z (tecla Q)


    if key == 263: # seta esquerda
        s_x += 0.01 
        s_y += 0.01
        s_z += 0.01
    if key == 262: # seta direita
        s_x -= 0.01 
        s_y -= 0.01
        s_z -= 0.01
    
    if key == 32: 
        if stopRotation == 0: 
            stopRotation = 1
        
glfw.set_key_callback(window,key_event)

In [423]:
glfw.show_window(window)

In [424]:
import math
d = 0.0
glEnable(GL_DEPTH_TEST) ### importante para 3D

from numpy import random


def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    ### apenas para visualizarmos o cubo rotacionando
    if stopRotation == 0:
        d += 0.01 # modifica o angulo de rotacao em cada iteracao
    cos_d = math.cos(d)
    sin_d = math.sin(d)
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    mat_rotation_z = np.matrix([    [cos_d, -sin_d, 0.0, 0.0], 
                                    [sin_d,  cos_d, 0.0, 0.0], 
                                    [0.0,      0.0, 1.0, 0.0], 
                                    [0.0,      0.0, 0.0, 1.0]], np.float32)
    
    mat_rotation_x = np.matrix([    [1.0,   0.0,    0.0, 0.0], 
                                    [0.0, cos_d, -sin_d, 0.0], 
                                    [0.0, sin_d,  cos_d, 0.0], 
                                    [0.0,   0.0,    0.0, 1.0]], np.float32)
    
    mat_rotation_y = np.matrix([    [cos_d,  0.0, sin_d, 0.0], 
                                    [0.0,    1.0,   0.0, 0.0], 
                                    [-sin_d, 0.0, cos_d, 0.0], 
                                    [0.0,    0.0,   0.0, 1.0]], np.float32)
    
    mat_translation = np.matrix([   [1.0, 0.0, 0.0, t_x], 
                                    [0.0, 1.0, 0.0, t_y], 
                                    [0.0, 0.0, 1.0, t_z], 
                                    [0.0, 0.0, 0.0, 1.0]], np.float32)

    mat_scale = np.matrix([   [s_x, 0.0, 0.0, 0.0], 
                              [0.0, s_y, 0.0, 0.0], 
                              [0.0, 0.0, s_z, 0.0], 
                              [0.0, 0.0, 0.0, 1.0]], np.float32)
    
    mat_rotation = mat_rotation_x @ mat_rotation_y @ mat_rotation_z
    mat_transform = mat_translation @ mat_scale @ mat_rotation



    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)

    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
   
    
    glUniform4f(loc_color, 0.874, 0.588, 0.752, 1.0) # vermelho
    glDrawArrays(GL_TRIANGLE_FAN, 0, num_vertices)
    glDrawArrays(GL_TRIANGLE_FAN, num_vertices, num_vertices*2)
    
    for triangle in range(0,len(vertices),3):
        R = 0.862
        G = 0.654
        B = 0.945
        glUniform4f(loc_color, R, G, B, 1.0)
        glDrawArrays(GL_TRIANGLES, triangle, 3)     


    
    glfw.swap_buffers(window)

glfw.terminate()